## Depression in Tweets

In [1]:
# import nltk library
import nltk; nltk.download('punkt')
from nltk import sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.tokenize.treebank import TreebankWordTokenizer

# import stopword libraries
nltk.download('stopwords'); from nltk.corpus import stopwords
from sklearn.feature_extraction import stop_words

# import other libraries
import pandas as pd
import numpy as np
import string
#from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import *
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import *
from sklearn.linear_model import LogisticRegression
from sklearn.grid_search import GridSearchCV

# import word embedding library
#import glove_helper

# import helper libraries
import collections
from common import utils, vocabulary

#display multiple results per cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#export models
from sklearn.externals import joblib

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/benthompson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/benthompson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/Users/benthompson/anaconda/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/benthompson/anaconda/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
#read in tweets
df = pd.DataFrame.from_csv('../depression_tweets.csv', header=None, parse_dates=True, infer_datetime_format=True)

In [3]:
#add index
df = df.reset_index()

#set column names
df.columns = ['date','tweet_id', 'handle', 'id', 'tweet', 'language', 'device', 'notes', 'notes_2']

In [4]:
#look at data
df.head(5)

,date,tweet_id,handle,id,tweet,language,device,notes,notes_2
0,2018-04-05 19:14:48,981973445616525312,Haldol,816793117785542656,Currently I am on 150 mg of hydroxyzine for in...,en,Twitter for iPhone,NaN,NaN
1,2018-04-05 19:14:48,981973444723064832,Rick O,3192532759,Integrated behavioral health for POLICE. Treat...,en,Twitter for iPhone,NaN,NaN
2,2018-04-05 19:14:47,981973443988996096,olivia 🧝🏽‍♀️ボス,1321438920,RT @DevinnJay: I won’t allow depression to fuc...,en,Twitter for iPhone,NaN,NaN
3,2018-04-05 19:14:47,981973443154505728,LeFrenchNeuropsy,2887994266,RT @LePsylab: For science ! Un questionnaire p...,fr,Twitter Web Client,NaN,NaN
4,2018-04-05 19:14:45,981973435705421826,GEEZ,311289251,I lost my brova I fell deep in depression!,en,Twitter for Android,NaN,NaN


In [5]:
#how man non-distinct tweets
len(df)

530773

In [6]:
#filter to english only
df = df[df['language'] == 'en']

In [7]:
#how many tweets now
len(df)

484663

In [8]:
#any users w/lots of tweets that might skew model?
#not any that seem too high
df['handle'].value_counts().head(5)

.                    1073
Aiden Hatfield        460
ً                     445
Documentary           313
In Music We Trust     312
Name: handle, dtype: int64

In [9]:
#how many distinct tweets
len(df.tweet.unique())

186362

In [10]:
#make distinct tweets the df
df = pd.DataFrame(df.tweet.unique())

In [11]:
#rename columns
df.columns = ['tweets']

In [12]:
#export sample to check quality
# pd.options.display.max_colwidth = 1000
# df_sample = df.sample(n=100)
# df_sample.to_csv('../sample_100_depression_tweets.csv')

In [13]:
#look up specific tweet
pd.options.display.max_colwidth = 10000
df.iloc[45055]

tweets    Y’all be using mental illness as a way to justify your bitchy behavior and it’s honestly a no from fucking me. Depr… https://t.co/YzlhNSV9RA
Name: 45055, dtype: object

In [14]:
#create column on 1's
x = [1]
x = x * len(df)
df['target'] = x

In [15]:
df.head(5)

,tweets,target
0,Currently I am on 150 mg of hydroxyzine for insomnia. As well as 300 mg Effexor XR and 4 mg Fanapt for Psychotic Depression.,1
1,Integrated behavioral health for POLICE. Treat mind &amp; body. The organizational &amp; operational stressors have long te… https://t.co/MnWDFqckQB,1
2,RT @DevinnJay: I won’t allow depression to fuck me up &amp; set me back. Nah not again.,1
3,I lost my brova I fell deep in depression!,1
4,RT @peachesfrfr: so there i am depression all over my titties,1


## Bring in random tweets

In [16]:
#read in tweets
df_2 = pd.DataFrame.from_csv('../random_tweets.csv', header=None)

In [17]:
#look at data
df_2.head()

""
0
True but she still cancelled tho.
"RT @roxxxdoxxx: when she said ""i gotta ask first"" i felt that 😫😅😂 https://t.co/BGPZqFLb9v"
appreciate this perfectly timed pic of me and catto pls https://t.co/GE5poooRcF
one of Beyoncé’s most underrated looks is the one from Jealous. don’t @ me
"""Once you create a system for censoring speech on the grounds that it is 'fake news' (even if it's parody, or sarca… https://t.co/EpuSUaK0UC"


In [18]:
#how many
len(df_2)

135177

In [19]:
#give index
df_2 = df_2.reset_index()

#give column name
df_2.columns = ['tweets']

In [20]:
#how many distinct tweets
len(df_2.tweets.unique())

111985

In [21]:
#Make dataframe of unique
df_2 = pd.DataFrame(df_2.tweets.unique())

#give column name
df_2.columns = ['tweets']

In [22]:
#make all tweets lowercase
df_2['tweets'] = df_2['tweets'].str.lower()
df_2.columns = ['tweets']

In [23]:
df_2.head()

,tweets
0,true but she still cancelled tho.
1,"rt @roxxxdoxxx: when she said ""i gotta ask first"" i felt that 😫😅😂 https://t.co/bgpzqflb9v"
2,appreciate this perfectly timed pic of me and catto pls https://t.co/ge5pooorcf
3,one of beyoncé’s most underrated looks is the one from jealous. don’t @ me
4,"""once you create a system for censoring speech on the grounds that it is 'fake news' (even if it's parody, or sarca… https://t.co/epusuak0uc"


In [24]:
#check for tweets that use depression
df_2[(df_2['tweets'].str.contains('depressed') | df_2['tweets'].str.contains('depression'))]

#drop them
df_2.drop(df_2[(df_2.tweets.str.contains('depressed')) | (df_2.tweets.str.contains('depression'))].index, inplace=True)

,tweets
1769,"rt @nickhansonmn: hey sorry i’ve been distant lately , i’m just super depressed about the current state of my life and didn’t wanna have to…"
1918,rt @matrix_reioaded: ahh i’m depressed... but all my teachers in elementary school said i was a special boy... how could this happen.. at o…
2568,rt @caucasianjames: on tinder depressed
2763,rt @softyoonle: hi this would really help me alot since i am battling depression and i need something to inspire me or help me 💞 if this fl…
3507,rt @iamsofiadg: philippines 🇵🇭 \nneed someone to talk to?\nsuicide/depression cellphone number \n\n0917-558-4673 📱\n\ncan you rt to potentially…
4159,rt @depressionnote: warning signs of depression ⚠️\n\n⚠️ low self-esteem\n⚠️ guilt\n⚠️ feeling hopeless\n⚠️ tiredness\n⚠️ loss of interest in thi…
4268,"rt @fuxksalliemae: alot of nigerians are struggling with depression, and much of it is financially and economically induced. don't just che…"
4547,rt @daitonreed: you do not have to come this hard on every song. aint no damn reason you needa make me depressed for no reason. https://t.c…
5501,rt @sionesnow: when i read the first sentence i thought there was a thing called “depression fcking” weh lol https://t.co/yup73ftjqc
6143,"i can't help the fact that i make all my characters depressed or not human, because i just realized that alexandria… https://t.co/scwoudncxy"


In [25]:
#recheck length
len(df_2)

111847

In [26]:
#export to check quality
# df_2_sample = df_2.sample(n=100)
# df_2_sample.to_csv('../sample_100_random_tweets.csv')

In [27]:
#column of 0's
x = 0
x = x * len(df_2)

df_2['target'] = x

In [28]:
#balance classes
df_3 = df.sample(n=len(df_2))

In [29]:
# df_3.head()

In [30]:
#combine dfs
df = pd.concat([df_3,df_2])

In [31]:
len(df)

223694

In [32]:
#preprocess tweets
example_text="""'RT @techreview: A neural network can 
detect depression and mania in bipolar subjects 
by analyzing how they hold and tap on their smartphone…'"""

# tokenize
def tokenize_text(input_text):
    """
    Args: 
    input_text: a string representing an 
    individual review
        
    Returns:
    input_token: a list containing stemmed 
    tokens, with punctutations removed, for 
    an individual review
        
    """
    input_tokens=[]
        
    # Split sentence
    sents=sent_tokenize(input_text)
            
    # Split word
    for sent in sents:
        input_tokens+=TreebankWordTokenizer().tokenize(sent)
        
    return input_tokens


# canonicalize
def canonicalize_tokens(input_tokens):
    """
    Args:
    input_tokens: a list containing tokenized 
    tokens for an individual review
    
    Returns:
    input_tokens: a list containing canonicalized 
    tokens for an individual review
    
    """
    input_tokens=utils.canonicalize_words(input_tokens)
    return input_tokens


# preprocessor 
def preprocessor(raw_text):
    """
    Args:
    raw_text: a string representing an
    individual review
    
    Returns:
    preprocessed_text: a string representing 
    a preprocessed individual review
    
    """
    # tokenize
    tokens=tokenize_text(raw_text)
    
    # canonicalize
    canonical_tokens=canonicalize_tokens(tokens)
    
    # rejoin string
    preprocessed_text=(" ").join(canonical_tokens) 
    return preprocessed_text

# example data
#input_tokens=tokenize_text(example_text)
#print(input_tokens)

#canonical_tokens=canonicalize_tokens(input_tokens)
#print(canonical_tokens)

preprocessed_text=preprocessor(example_text) 
print(preprocessed_text)

'rt @ techreview : a neural network can detect depression and mania in bipolar subjects by analyzing how they hold and tap on their smartphone… '


In [33]:
# examine stopwords

# sklearn stopwords (frozenset)
sklearn_stopwords=stop_words.ENGLISH_STOP_WORDS
print("number of sklearn stopwords: %d" %(len(sklearn_stopwords)))
#print(sklearn_stopwords)

# nltk stopwords (list)
nltk_stopwords=stopwords.words("english")
print("number of nltk stopwords: %d" %(len(nltk_stopwords)))
#print(nltk_stopwords)

# combined sklearn, nltk, other stopwords (set)
total_stopwords=set(list(sklearn_stopwords.difference(set(nltk_stopwords)))+nltk_stopwords)

other_stopwords=["DG", "DGDG", "@", "rt", "'rt", "'", ":", "depression", "depressed"]
for w in other_stopwords:
    total_stopwords.add(w)
    
print("number of total stopwords: %d" %(len(total_stopwords)))

number of sklearn stopwords: 318
number of nltk stopwords: 179
number of total stopwords: 387


In [34]:
#look at review w/o stop words
new_review = []
for i in preprocessed_text.split():
    if i in total_stopwords:
        continue
    else:
        new_review.append(i)
        
print(new_review)

['techreview', 'neural', 'network', 'detect', 'mania', 'bipolar', 'subjects', 'analyzing', 'hold', 'tap', 'smartphone…']


In [35]:
#reset index
df = df.reset_index(drop=True)

In [36]:
#split into test, train before sampling to belance
# using recoded labels
#create train, test data
df['is_train'] = np.random.uniform(0,1, len(df)) <= .8

train_data, test_data = df[df['is_train'] == True], df[df['is_train'] == False]

# examine train, test shapes
print("train, test set size: %d, %d" %(len(train_data), len(test_data))) # train_data: 129023, test_data: 32256
print("")

# examine train set examples
print("example:")
print("tweet: %s" %(train_data.get_value(5,'tweets')))
print("label: %s" %(train_data.get_value(5,'target')))

train, test set size: 179044, 44650

example:
tweet: RT @EboneeDavis: Stop making depression an aesthetic.
label: 1


In [37]:
#check class balance
train_data['target'].value_counts()

1    89604
0    89440
Name: target, dtype: int64

In [38]:
print("example:")
print("tweet: %s" %(train_data.get_value(31,'tweets')))
print("label: %s" %(train_data.get_value(31,'target')))

example:
tweet: Having anxiety/depression  keeping a positive mind  and shit constantly going wrong is the worst battle ever
label: 1


## Logistic Regression

In [39]:
#build tf-idf model
vec=TfidfVectorizer(preprocessor=preprocessor, ngram_range=(1,3), stop_words=total_stopwords, max_features=10000)
vec_train_data=vec.fit_transform(train_data['tweets']) 
vec_test_data=vec.transform(test_data['tweets']) 

In [40]:
# train Logistic Regression
logit=LogisticRegression(penalty='l2')
logit.fit(vec_train_data, train_data['target'])
pred_labels=logit.predict(vec_test_data)
    
# assess model
f1=f1_score(test_data['target'], pred_labels, average="weighted") 
accuracy=accuracy_score(test_data['target'], pred_labels)
confusion=confusion_matrix(test_data['target'], pred_labels)
print("logistic regression f1 score: %.3f" %(f1))
print("logistic regression accuracy score: %.3f" %(accuracy))
print("logistic regression confusion matrix:")
print(confusion)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

logistic regression f1 score: 0.819
logistic regression accuracy score: 0.819
logistic regression confusion matrix:
[[19295  3112]
 [ 4950 17293]]


In [41]:
#try Keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

Using TensorFlow backend.
/Users/benthompson/anaconda/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [47]:
#build tf-idf model
vec_2=TfidfVectorizer(preprocessor=preprocessor, stop_words=total_stopwords, max_features=10000)
tfidf_mat = vec_2.fit_transform(train_data['tweets']).toarray()

In [50]:
# create the model
top_words = 10000
embedding_size = 32

model = Sequential()
model.add(Embedding(top_words, embedding_size, input_shape=(10000,)))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 10000, 32)         320000    
_________________________________________________________________
flatten_2 (Flatten)          (None, 320000)            0         
_________________________________________________________________
dense_3 (Dense)              (None, 250)               80000250  
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 251       
Total params: 80,320,501
Trainable params: 80,320,501
Non-trainable params: 0
_________________________________________________________________
None


In [51]:
# Fit the model
epochs=3
batch_size=128

history = model.fit(tfidf_mat, train_data['target'],
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1)

# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 161139 samples, validate on 17905 samples
Epoch 1/3
161139/161139 [==============================] - 3860s 24ms/step - loss: 7.0727 - acc: 0.5561 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 2/3
161139/161139 [==============================] - 3884s 24ms/step - loss: 7.0774 - acc: 0.5561 - val_loss: 15.9424 - val_acc: 0.0000e+00
Epoch 3/3
 75392/161139 [=============>................] - ETA: 33:50 - loss: 7.0382 - acc: 0.5585

KeyboardInterrupt: 

In [51]:
#get top words
#look at top 5 weights for each class
#get coefficients for all features
coef_sq = logit.coef_

#get index of top 5 absolute values for each class
weight_indx = np.argsort(coef_sq)[:, -20:]

#flatten so can use to look up wieghts
weight_indx = weight_indx.flatten()

#get coefficients based on index
weights = coef_sq[:, weight_indx]
 
#get words that match weights based on index
vocab = np.array(vec.get_feature_names())[weight_indx]

# make table
df = pd.DataFrame({'Weights of words that predict depression': weights[0]}
                  , index=vocab)
df

,Weights of words that predict depression
naps,5.900655
battling,6.385033
cures,6.390737
suicidal,6.418737
nap,6.499706
sadness,6.534502
clinical,6.537225
mental,6.697283
postpartum,6.738400
suffering,6.996678


In [126]:
#try to make up an example journal
journal = """Today was wonderful. I had a strange interaction at the store. 
The cashier seemed irratated. I'm not sure what's going on but it makes me feel weird"""

#score test journal
vec_test_example=vec.transform([journal]) 
print("probability of class 0 and 1: ",logit.predict_proba(vec_test_example))

#get words and weights from test journal
word_idx = np.nonzero(vec_test_example)[1]
vocab = np.array(vec.get_feature_names())[word_idx]
weights = coef_sq[:, word_idx]
df = pd.DataFrame({'Weights of words in sample Journal': weights[0]}
                  , index=vocab)
df.sort_values(by='Weights of words in sample Journal')

probability of class 0 and 1:  [[ 0.43736519  0.56263481]]


,Weights of words in sample Journal
store,-1.171633
sure,-0.457797
interaction,-0.419084
wonderful,-0.374465
strange,0.326909
weird,0.392027
today,0.923644
makes feel,1.083485
going,1.660564
makes,1.798566


In [127]:
#export tfidf model
tfidf_file = 'tfidf_exported_model'
joblib.dump(vec, tfidf_file)

['tfidf_exported_model']

In [128]:
#export logistic regression
logistic_regression_file = 'logistic_regression_model'
joblib.dump(logit, logistic_regression_file)

['logistic_regression_model']

In [129]:
#test out exported models against prev sample journal
loaded_tfidf = joblib.load('tfidf_exported_model')
loaded_lr = joblib.load('logistic_regression_model')

#score test journal
export_test_example=loaded_tfidf.transform([journal]) 
print("probability of class 0 and 1: ",loaded_lr.predict_proba(export_test_example))


probability of class 0 and 1:  [[ 0.43736519  0.56263481]]
